In [1]:
import os, numpy as np

In [142]:
class Lab1:
    def __init__(self, n):
        self.n = n


    def get_X(self, begin, end, size) : return np.random.normal(begin, end, size = size)


    def get_a(self, X) : return np.divide(np.sum(X), len(X))


    def get_sigma(self, X) : return np.sqrt(np.divide(np.sum((X - self.get_a(X)) ** 2), len(X)))


    def solve_A(self, z, expected):
        for n_i in self.n:
            X = self.get_X(expected, 1, n_i)
            a = self.get_a(X)
            sigma = self.get_sigma(X)
            l = a - z * sigma / np.sqrt(n_i - 1); r = a + z * sigma / np.sqrt(n_i - 1)
#             print(type(left), type(right), type(expected))
#             print(left)
            print(f'n = {n_i}: ',\
                  f'\n\t{expected} попал в интервал ({l}, {r})\n' if expected > l and expected < r \
                  else f'\n\t{expected} попал в интервал ({l}, {r})\n')
    
    
    def solve_B(self, z, expected):
        for n_i in self.n:
            X = self.get_X(expected, 1, n_i)
            a = self.get_a(X)
            sigma = self.get_sigma(X)
            l = a - z * sigma / np.sqrt(n_i) ;r = a + z * sigma / np.sqrt(n_i)
#             print(type(left), type(right), type(expected))
#             print(left)
            print(f'n = {n_i}: ',\
                  f'\n\t{expected} попал в интервал ({l}, {r})\n' if expected > l and expected < r \
                  else f'\n\t{expected} попал в интервал ({l}, {r})\n')
    
    
    def solve_C(self, z, variety):
        for n_i, z_i in zip(self.n, z):
            X = self.get_X(0, variety, n_i)
            sigma = self.get_sigma(X)
            l = n_i * sigma / z_i[1]; r = n_i * sigma / z_i[0]
            print(f'n = {n_i}: ',\
                  f'\n\t{variety} попал в интервал ({l}, {r})\n' if variety > l and variety < r \
                  else f'\n\t{variety} попал в интервал ({l}, {r})\n')

In [147]:
L1 = Lab1(np.array([100, 10000, 1000000]))

In [148]:
L1.solve_A(2.575, .0)

n = 100:  
	0.0 попал в интервал (-0.2437310729083624, 0.2528469787863499)

n = 10000:  
	0.0 попал в интервал (-0.018639923797353707, 0.03317453107445203)

n = 1000000:  
	0.0 попал в интервал (-0.005457966799697012, -0.0003012966730975273)



In [149]:
L1.solve_B(2.575, .0)

n = 100:  
	0.0 попал в интервал (-0.13087591720460015, 0.3787049061350488)

n = 10000:  
	0.0 попал в интервал (-0.01774200114171138, 0.033967753412141775)

n = 1000000:  
	0.0 попал в интервал (-0.004403594269308467, 0.0007403153503507183)



In [151]:
z = [(66.5101053, 138.9867835), (9638.498197, 10367.01484), (996359.9858, 1003645.527)]

L1.solve_C(z, 1.)

n = 100:  
	1.0 попал в интервал (0.7737100566525804, 1.6168290765846212)

n = 10000:  
	1.0 попал в интервал (0.9490237171668484, 1.0207547647249593)

n = 1000000:  
	1.0 попал в интервал (0.9959901730679701, 1.0032730101390068)

